In [0]:
def build_w(query_id):
    ans=defaultdict(float)
    t=query_ext[query_id]
    s=sum(map(lambda x:x[0]*x[0],t[:15]))
    s1=sum(map(lambda x:x[0]*x[0],t[:5]))
    s2=sum(map(lambda x:x[0]*x[0],t[5:10]))
    s3=sum(map(lambda x:x[0]*x[0],t[10:15]))
    v1=s1/s
    v2=s2/s
    v3=s3/s

    for w,word in t[:5]:
        ans[word]=w*w/s1+v1
    for w,word in t[5:10]:
        ans[word]=w*w/s2+v2
    for w,word in t[10:15]:
        ans[word]=w*w/s3+v3
    
    low=t[15][0]*t[15][0]/s3+v3
    for w,word in t[15:]:
        ans[word]=low
    
    for word in set(map(lambda word:add_cash(word),
                        get_query[query_id].split())):
        ans[word]=max(1,ans[word]+1)
    
    N=len(get_id)
    idf=0
    for word in ans:
        idf+=RIDF(word,N,0,0.5,2)*ans[word]
    return ans,idf

In [0]:
def group_by_doc(l,w):
    w_bert=[1,0.8,0.7,0.6,0.5]
    w_idf =[1,0.8,0.7,0.6,0.5]

    W=defaultdict(lambda:defaultdict(float,w))

    tmp=defaultdict(lambda :[[],0,0,0])

    for x in l:
        new_w=W[x.doc_id]
        t=tmp[x.doc_id]

        while len(t[0])>=len(w_bert):
            w_bert.append(w_bert[-1]/2)
        while len(t[0])>=len(w_idf):
            w_idf.append(w_idf[-1]/2)

        c_bert=w_bert[len(t[0])]
        c_idf=w_idf[len(t[0])]

        t[0].append(x.sent)
        heap=[(0,'')]*3
        for word in set(map(add_cash,x.sent.split())):
            heapq.heappushpop(heap,(RIDF(word,len(get_id),0,0.5,2)*new_w[word],word))
        idf=reduce(lambda x,y:x+y[0],heap,0)
        for _,word in heap:
            new_w[word]/=2.25


        t[1]+=x.score_bert*c_bert
        t[2]+=x.pos*c_bert
        t[3]+=idf*c_idf
    
    ans=[Passage_idf(t[0],t[1],t[2],doc_id,t[3]) 
    for doc_id,t in tmp.items()]

    return ans

In [0]:
def st(tmp):
    t=np.array([x.score_bert for x in tmp])
    std=np.std(t)
    mean=np.mean(t)
    tmp=[Passage_idf(x.sent,y,x.pos,x.doc_id,x.score_idf) for x,y in zip(tmp,(t-mean)/std)]

    t=np.array([x.score_idf for x in tmp])
    std=np.std(t)
    mean=np.mean(t)
    tmp=[Passage_idf(x.sent,x.score_bert,x.pos,x.doc_id,y) for x,y in zip(tmp,(t-mean)/std)]

    min_bert=reduce(lambda x,y:min(x,y.score_bert),tmp,1000)
    min_idf=reduce(lambda x,y:min(x,y.score_idf),tmp,1000)
    tmp=[Passage_idf(x.sent,x.score_bert-min_bert,x.pos,x.doc_id,x.score_idf-min_idf) for x in tmp]

    return tmp

In [0]:
def comp_idf(sent,w):
    words=set(map(lambda word:add_cash(word),
                  sent.split()))
    heap=[-1]*3
    for word in words:
        heapq.heappushpop(heap,RIDF(word,len(get_id),0,0.5,2)*w[word])
    return sum(heap)

In [0]:
def RIDF(word,N,R,k,l):
    n=IDF_lem[word]
    if R:
        r=r_counter[word]
        return log(1 + (r+k)*(N-n-R+r+k)/(n-r+k)/(R-r+k))
    else:
        if n==0:
            return 0
        return log(l+(N-n)/n)

In [0]:
Passage_idf=namedtuple('Passage_idf',['sent','score_bert','pos','doc_id','score_idf'])
def full(query_id,all_scores):
    tmp=all_scores[query_id]

    w,_=build_w(query_id)
    tmp=list(map(lambda x:Passage_idf(*x,comp_idf(x.sent,w)),tmp))

    
    tmp=st(tmp)

    tmp=group_by_doc(sorted(tmp,key=lambda x:-1/(1/(x.score_bert+1e-5) + 1/(x.score_idf +1e-5))-0.15*x.pos),w)
    tmp=st(tmp)
    return tmp